In [1]:
from datetime import datetime
from IPython.core.debugger import set_trace
import math
import pandas as pd
import sqlite3

In [2]:
DB = './db.sqlite'
TABLE = 'Philosopher'
COLUMNS = 'birthDate', 'deathDate'

In [3]:
def get_df():
    try:
        conn = sqlite3.connect(DB)
        df = pd.read_sql(f'SELECT * FROM {TABLE}', conn)
    finally:
        conn.close()
    return df

df = get_df()
df.tail(20)

,wikiPageID,abstract,gender,birthDate,deathDate,name
1893,244468,"Zhu Xi or Chu Hsi (Chinese: 朱熹, October 18, 11...",male,1130-10-18,1200-04-23,Zhu Xi
1894,166152,"Zhuang Zhou, often known as Zhuangzi (""Master ...",male,-0364-01-01,-0289-01-01,Zhuang Zhou (莊周)
1895,245077,Zoilus or Zoilos (Greek: Ζωΐλος; c. 400 – 320 ...,male,-0400-01-01,-0320-01-01,Zoilus
1896,31848187,Zoltán Dörnyei (Hungarian pronunciation: [ˈzol...,male,1960-03-11,None,Zoltán Dörnyei
1897,666212,Zygmunt Bauman (born 19 November 1925) is a Po...,male,1925-11-19,None,Zygmunt Bauman
1898,2173176,Ágnes Heller (born 12 May 1929) is a Hungarian...,female,1929-05-12,None,Ágnes Heller
1899,1032341,Élie Halévy (6 September 1870 – 21 August 1937...,male,1870-09-06,1937-08-21,Élie Halévy
1900,1582235,Émile Armand (pseudonym of Ernest-Lucien Juin ...,male,1872-03-26,1962-02-19,Ernest-Lucien Juin Armand
1901,463542,Étienne Émile Marie Boutroux (French: [butʁu];...,male,1845-07-28,1921-11-22,Émile Boutroux
1902,765362,Émile-Auguste Chartier (French: [ʃaʁtje]; 3 Ma...,male,1868-03-03,1951-06-02,Émile Chartier


In [4]:
def date_to_jd(year,month,day):
    if month == 1 or month == 2:
        yearp = year - 1
        monthp = month + 12
    else:
        yearp = year
        monthp = month
    
    # this checks where we are in relation to October 15, 1582, the beginning
    # of the Gregorian calendar.
    if ((year < 1582) or
        (year == 1582 and month < 10) or
        (year == 1582 and month == 10 and day < 15)):
        # before start of Gregorian calendar
        B = 0
    else:
        # after start of Gregorian calendar
        A = math.trunc(yearp / 100.)
        B = 2 - A + math.trunc(A / 4.)
        
    if yearp < 0:
        C = math.trunc((365.25 * yearp) - 0.75)
    else:
        C = math.trunc(365.25 * yearp)
        
    D = math.trunc(30.6001 * (monthp + 1))
    
    jd = B + C + D + day + 1720994.5
    
    return jd

def fix_dates(df):
    def fix_date(s):
        if s is not None:
            bce = False
            if s[0] == '-':
                bce = True
                s = s[1:]
            year, month, day = s.split('-')
            year, month, day = int(year), int(month), int(day)
            if month == 0:
                month = 1
            if day == 0:
                day = 1
            if bce:
                year = -year
            return date_to_jd(year, month, day)

    df = df.copy(deep=True)
    for col in COLUMNS:
        df[col] = df[col].apply(fix_date)
    df.sort_values(by='birthDate')
    return df
        
df_ = fix_dates(df)
df_.tail(20)

,wikiPageID,abstract,gender,birthDate,deathDate,name
1893,244468,"Zhu Xi or Chu Hsi (Chinese: 朱熹, October 18, 11...",male,2134080.5,2159470.5,Zhu Xi
1894,166152,"Zhuang Zhou, often known as Zhuangzi (""Master ...",male,1588106.5,1615500.5,Zhuang Zhou (莊周)
1895,245077,Zoilus or Zoilos (Greek: Ζωΐλος; c. 400 – 320 ...,male,1574957.5,1604177.5,Zoilus
1896,31848187,Zoltán Dörnyei (Hungarian pronunciation: [ˈzol...,male,2437004.5,NaN,Zoltán Dörnyei
1897,666212,Zygmunt Bauman (born 19 November 1925) is a Po...,male,2424473.5,NaN,Zygmunt Bauman
1898,2173176,Ágnes Heller (born 12 May 1929) is a Hungarian...,female,2425743.5,NaN,Ágnes Heller
1899,1032341,Élie Halévy (6 September 1870 – 21 August 1937...,male,2404311.5,2428766.5,Élie Halévy
1900,1582235,Émile Armand (pseudonym of Ernest-Lucien Juin ...,male,2404878.5,2437714.5,Ernest-Lucien Juin Armand
1901,463542,Étienne Émile Marie Boutroux (French: [butʁu];...,male,2395140.5,2423015.5,Émile Boutroux
1902,765362,Émile-Auguste Chartier (French: [ʃaʁtje]; 3 Ma...,male,2403394.5,2433799.5,Émile Chartier


In [5]:
def save(df):
    try:
        conn = sqlite3.connect(DB)
        conn.execute(f"DELETE FROM {TABLE}")
        df.to_sql(TABLE, conn, if_exists='append', index=False)
        conn.commit()
    finally:
        conn.close()
        
save(df_)